# routers

> Contains all the routes. (routers subfoler in tut; nbdev did not transfer a subfolder under nbs to the package folder)

In [3]:
#| default_exp routers

In [4]:
!which python

/home/uts/aishiftscheduler/.venv/bin/python


In [5]:
!python --version

Python 3.10.12


In [6]:
# Set up autoreload
%load_ext autoreload
%autoreload 2

In [7]:
#| hide
from nbdev.showdoc import *

In [8]:
# !pip install psycopg2

In [10]:
#| export
from typing import List
from fastapi import Response, status, HTTPException, Depends, APIRouter
from fastapi.security.oauth2 import OAuth2PasswordRequestForm
# from pydantic import BaseModel
# # from passlib.context import CryptContext
# from fastapi.middleware.cors import CORSMiddleware
# from fastapi.params import Body
# from random import randrange
# import psycopg2
# from psycopg2.extras import RealDictCursor

# ## sqlalchemy and ORM (database.py from tut)
# from sqlalchemy import create_engine
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
# SQLALCHEMY_DATABASE_URL = 'postgresql://postgres:p@localhost/sai_db2'
# engine = create_engine(SQLALCHEMY_DATABASE_URL)
# SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
# Base = declarative_base()

import aishiftscheduler.dbmodels as dbm
import aishiftscheduler.schemas as sch
import aishiftscheduler.utils as utl
import aishiftscheduler.oauth2 as oa2
from aishiftscheduler.database import get_db

/home/uts/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
/home/uts/aishiftscheduler/aishiftscheduler/best_theta_data.xlsx
/home/uts/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png

len(thNAMES)=5
thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']


## Authentication (tut auth.py)

In [ ]:
#| export
auth_router = APIRouter(tags=['Authentication'])

In [ ]:
#| export
@auth_router.post('/login', response_model=sch.Token)
# def login(user_credentials: sch.UserLogin, db: Session=Depends(get_db)):
def login(user_credentials: OAuth2PasswordRequestForm=Depends(), db: Session=Depends(get_db)):
    # user = db.query(dbm.User).filter(dbm.User.email == user_credentials.email).first()
    user = db.query(dbm.User).filter(dbm.User.email == user_credentials.username).first()

    if not user:
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN, 
            # detail=f"Invalid Credentials"
            detail=f"not user !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
        )

    if not utl.verify(user_credentials.password, user.password):
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN, 
            # detail=f"Invalid Credentials"
            detail=f"fail verify !!!!!!!!!!!!!!!!!!!!!!!!!!!!"
        )
    
    ## create a token
    ## return token
    access_token = oa2.create_access_token(data={"user_id": user.id})
    return {"access_token": access_token, "token_type": "bearer"}

In [ ]:
#| export
userinput_router = APIRouter(
    prefix="/userinputs",
    tags=['UsersInputs']
)

## UserInput routes

In [ ]:
#| export
# @app.get("/userinputs", response_model=List[sch.UserInput])
@userinput_router.get("/", response_model=List[sch.UserInput])
def get_userinputs(db: Session=Depends(get_db), user_id: int=Depends(oa2.get_current_user)):
    userinputs = db.query(dbm.UserInput).all()
    return userinputs

In [ ]:
#| export
@userinput_router.post("/", status_code=status.HTTP_201_CREATED, response_model=sch.UserInput)
# def create_userinputs(userinput: sch.UserInputCreate, db: Session = Depends(get_db), get_current_user: int=Depends(oa2.get_current_user)):
# def create_userinputs(userinput: sch.UserInputCreate, db: Session = Depends(get_db), user_id: int=Depends(oa2.get_current_user)):
def create_userinputs(userinput: sch.UserInputCreate, db: Session = Depends(get_db), current_user: int=Depends(oa2.get_current_user)):  
    # print(user_id)
    # print(current_user.email)
    # new_userinput = dbm.UserInput(**userinput.model_dump())
    new_userinput = dbm.UserInput(user_id=current_user.id, **userinput.model_dump())
    db.add(new_userinput)
    db.commit()
    db.refresh(new_userinput)
    return new_userinput

In [ ]:
#| export
@userinput_router.get("/{id}", response_model=sch.UserInput)
# def get_userinput(id: int, db: Session = Depends(get_db), user_id: int=Depends(oa2.get_current_user)):
def get_userinput(id: int, db: Session = Depends(get_db), current_user: int=Depends(oa2.get_current_user)):
    userinput = db.query(dbm.UserInput).filter(dbm.UserInput.id == id).first()
    if not userinput:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail=f"userinput with id: {id} was not found")
    return userinput

In [ ]:
#| export
@userinput_router.delete("/{id}", status_code=status.HTTP_204_NO_CONTENT)
# def delete_userinput(id: int, db: Session = Depends(get_db), user_id: int=Depends(oa2.get_current_user)):
def delete_userinput(id: int, db: Session = Depends(get_db), current_user: int=Depends(oa2.get_current_user)):    
    userinput = db.query(dbm.UserInput).filter(dbm.UserInput.id == id)
    if userinput.first() == None:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail=f"userinput with id: {id} does not exist")
    userinput.delete(synchronize_session=False)
    db.commit()
    return Response(status_code=status.HTTP_204_NO_CONTENT)


In [ ]:
#| export
@userinput_router.put("/{id}", response_model=sch.UserInput)
# def update_userinput(id: int, updated_userinput: sch.UserInputUpdate, db: Session=Depends(get_db), user_id: int=Depends(oa2.get_current_user)):
def update_userinput(id: int, updated_userinput: sch.UserInputUpdate, db: Session=Depends(get_db), current_user: int=Depends(oa2.get_current_user)):
    userinput_query = db.query(dbm.UserInput).filter(dbm.UserInput.id == id)
    userinput = userinput_query.first()
    if userinput == None:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND, 
            detail=f"userinput with id: {id} does not exist")    
    userinput_query.update(updated_userinput.model_dump(), synchronize_session=False)
    db.commit()
    return userinput_query.first()

## User routes

In [ ]:
#| export
user_router = APIRouter(
    prefix="/users",
    tags=['Users']
)

In [ ]:
#| export
## users
@user_router.post("/", status_code=status.HTTP_201_CREATED, response_model=sch.UserOut)
def create_user(user: sch.UserCreate, db: Session=Depends(get_db)):

    # has the password - user.password
    # hashed_password = pwd_context.hash(user.password)
    hashed_password = utl.hash(user.password)
    user.password = hashed_password

    new_user = dbm.User(**user.model_dump())
    db.add(new_user)
    db.commit()
    db.refresh(new_user)
    return new_user